## Stock AI assistant

#### Important Features of Stock AI assistant

### 💬 **Interactive Chat Interface**
- Real-time conversation via Gradio.
- Provides clear, polite responses based on user input.

### 3rd party APIs 
- Reading stock market data from yahoo finance
- Downloading the data for specific time and working on them
- Using daily closing price
  
### 🛠️ **Modular Tool Support**
- Integrated tools for:
  - Buying stock
  - Selling stock
  - Returning the account balance 
  - Adding cash to account
- Easily extensible for future features.

### 🛡️ **Error Handling**
- Validates user inputs for menu.
- Graceful error messages for smooth user experience.
- There are some logical errors in StockSimulator class. Also, I tried to leave some tasks to ChatGPT to see how it handles some possible errors, but I realized ChatGPT is not able to handle all the errors because it's not good to do math or do comparison for numerical values and dates. In non-numerical  case, it has good handling of the errors specially when the user input 
##### comparison, some results are not correct. 



In [1]:
# !pip install yfinance

In [2]:
import os
import json
import random
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"  # or "gpt-3.5-turbo", etc.
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [36]:
import yfinance as yf
import matplotlib.pyplot as plt

class StockSimulator:
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers  # List of tickers
        self.start_date = start_date
        self.end_date = end_date
        self.data = {}
        self.cash = 10000  # Starting cash balance
        self.portfolio = {ticker: 0 for ticker in tickers}  # Holdings per ticker
        self.history = []  # Transaction history

    def get_cash(self):
        return f"Your account's cash balance is {self.cash}"

    def add_cash(self, value):
        if isinstance(value, str):
            value = float(value)
        self.cash += value
        return f"${value} is added as cash to the account. Your account's cash is {self.cash}."

    def fetch_data(self, ticker):
        # Download historical stock data for all tickers
        # for ticker in self.tickers:
        stock_data = yf.download(ticker, start=self.start_date, end=self.end_date)
        self.data[ticker] = stock_data['Close']
        if self.data[ticker].empty:
            print(f"No data found for {ticker}.")
        else:
            print(f"Data fetched for {ticker}.")
                
    def get_price(self, date, ticker):
        if ticker not in self.data:
            self.fetch_data(ticker)#self.data[ticker] = yf.download(ticker, self.start_date, end=self.end_date)
        if self.data[ticker].empty:
            return f"No data available for {ticker}. Cannot buy."
            
        return f" {ticker} price is {self.data[ticker].loc[date].item()}"
        

    def buy(self, date, ticker, shares):
        if ticker not in self.data or self.data[ticker].empty:
            self.data[ticker] = yf.download(ticker, start=self.start_date, end=self.end_date)
            if self.data[ticker].empty:
                return f"No data available for {ticker}. Cannot buy."
        
        if isinstance(shares, str):
            shares = int(shares)

        
        price = self.data[ticker].loc[date].item()
        cost = shares * price
        if self.cash >= cost:
            self.cash -= cost
            self.portfolio[ticker] += shares
            self.history.append(f"Bought {shares} shares of {ticker} at ${price:.2f} on {date}")
            return f"Successfully bought {shares} shares of {ticker} at ${price:.2f}"
        else:
           return "Insufficient cash to buy shares."

    def sell(self, date, ticker, shares):
        if ticker not in self.data or self.data[ticker].empty:
            print(f"No data available for {ticker}. Cannot sell.")
            return
        
        if isinstance(shares, str):
            shares = int(shares)
        
        price = self.data[ticker].loc[date].item()
        if self.portfolio.get(ticker, 0) >= shares:
            self.portfolio[ticker] -= shares
            self.cash += shares * price
            self.history.append(f"Sold {shares} shares of {ticker} at ${price:.2f} on {date}")
            return f"Successfully sold {shares} shares of {ticker} at ${price:.2f}"
        else:
            return "Not enough shares to sell."

    def portfolio_value(self, date):
        total_value = self.cash
        for ticker, shares in self.portfolio.items():
            if ticker in self.data and not self.data[ticker].empty:
                price = self.data[ticker].loc[date].item()
                total_value += shares * price
        return f"The total balance in your account is {total_value}"

    def plot_performance(self):
        # Plot portfolio value over time
        dates = list(self.data.values())[0].index  # Use index from any stock data
        portfolio_values = [self.portfolio_value(date) for date in dates]
        
        plt.figure(figsize=(12, 6))
        for ticker, stock_data in self.data.items():
            plt.plot(stock_data.index, stock_data, label=f"{ticker} Price")
        plt.plot(dates, portfolio_values, label="Portfolio Value", linestyle="--", color="black")
        plt.xlabel("Date")
        plt.ylabel("Value")
        plt.title("Portfolio Performance")
        plt.legend()
        plt.show()

# Example Usage
sim = StockSimulator(tickers=["AAPL", "MSFT", "GOOGL"], start_date="2022-01-01", end_date="2022-12-31")
sim.fetch_data("AAPL")
sim.buy("2022-01-10", "AAPL", 10)  # Buy 10 shares of AAPL
# sim.buy("2022-01-12", "MSFT", 5)  # Buy 5 shares of MSFT
sim.sell("2022-02-15", "AAPL", 5)  # Sell 5 shares of AAPL
# sim.sell("2022-03-01", "MSFT", 2)  # Sell 2 shares of MSFT
# print(f"Portfolio Value on 2022-12-31: ${sim.portfolio_value('2022-12-30'):.2f}")
# sim.plot_performance()


[*********************100%***********************]  1 of 1 completed

Data fetched for AAPL.


'Successfully sold 5 shares of AAPL at $170.22'

In [12]:
isinstance('2', str)

True

In [31]:
system_message = ("You are a helpful assitant in stock market. You process the order of users. The users request buy and sell orders.\n"
                  "When a user wants to use you, follow these steps:\n"
                  "1- Show a main menu to user to choose. The menu shows the following options to user as a numbered list: [Stock price check, Balance, Buy, Sell,  Cash, Add Cash].\n"
                  "User should choose one option and if the user input is not the number correspond to the option inside the menu, show a message and ask the user to choose a correct number\n"
                  "2- if the user chooses Stock price check, ask the user to enter the name of stock\n"
                  "3- Call the function get_price function ask the user to enter stock's or ticker's name, and the date. The name and date are used as parameters of the get_price function\n"
                  "   - if the output is empty, say:'this stock is not found in the database'. Then return to step 1 to show the menu again and ask the user to choose a new option.\n"
                  "   - if the output is not empty, show a message to the userthat the stock is available and can be traded\n" 
                  "   - use the name of stock and the date to call the function get_price and show its output to user. Then return to the menu in step 1\n"
                  "4- if use chooses Buy option, ask the user the date of purchase, the name of stock, and the number of shares to buy. Use this information to call buy function\n"
                  "   -if the function returns some errors related to problem to complete the buy order like lack of cash, wrong stock's name and so on, show a message based on the error and go back to main menu in step 1\n"
                  "   - if there is no errors, show a confirmation message as output of the function\n"
                  "5-do the same for Sell option in main menu after the sell function if it is the user choice.\n"
                  "   - ask the name of stock, the number of shares and the date. Pay attention that the date of sell must not be ealier than buy date.\n"
                  "   - If something is wrong based on the output of sell function, show the message and if everything is okay, just show  the confirmation message.\n"
                  "6- the balance option in main menu calls portfolio_value function and asks the user to enter the date.\n"
                  "7-if the user chooses Cash option from main menu, calls the get_cash function to show the cash value in the user's account. This cash value is used to give the power purchase to user\n"
                  "8- if the user chooses add cash option, call the function add_cash and ask the user the amount of cash he/she wants to add to the account\n"
                  "If you don't know something, say so.\n"
                 "Keep answers short and courteous.\n"
                  "IMPORTANT \n"
                  "show to user that the maximum date is 2022-02-18 in the first message that you show to the user. If the user enters a date after 2022-02-18, ask the user to enter the date again \n"
                  )

In [14]:
###############################################################################
# 4) Tools JSON Schemas
##############################################################################
cash_function = {
    "name": "get_cash",
    "description": "Get the amount of cash in account or portfolio ",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
    },
}
#  buy(self, date, ticker, shares)
buy_function = {
    "name": "buy",
    "description": "Get the amount of shares to buy an stock for the user's account",
    "parameters": {
        "type": "object",
        "properties": {
            "date": {
                "type": "string",
                "description": "Date of buying a stock. The format should be YYYY-MM-DD",
            },
            "ticker": {
                "type": "string",
                "description":"the name of the stock or ticker",
            },
            "shares":{
                "type":"string",
                "description":"the number of share to purchase from a stock and adding it to the account",
            },
        },
        "required": ["date", "ticker", "shares"],
    },
}

# def sell(self, date, ticker, shares)
sell_function = {
    "name": "sell",
    "description": "Get the amount of shares to sell from the user's account",
    "parameters": {
        "type": "object",
        "properties": {
            "date": {
                "type":"string",
                "description":"date of selling stock. The format should be YYYY-MM_DD",
            },
            "ticker": {
                "type": "string",
                "description":"the name of the stock or ticker to sell",
            },
            "shares":{
                "type":"string",
                "description":"the number of share to sell from a ticker/stock and subtracting it from the account",
            },
        },
        "required": ["date", "ticker", "shares"],
    },
}

# def get_price(self, date, ticker):
get_price_function = {
    "name": "get_price",
    "description": "Get the price of stock at specific date",
    "parameters": {
        "type": "object",
        "properties": {
            "date": {
                "type":"string",
                "description":"date with format YYYY-MM_DD, which specifies the price point of the stock in dataset based on the date",
            },
            "ticker": {
                "type": "string",
                "description":"the name of the stock or ticker to show the price",
            },
        },
        "required": ["date", "ticker"],
    },
}

# def portfolio_value(self, date)
get_balance_function = {
    "name": "portfolio_value",
    "description": "shows the value of account, which is the sum of cash and value of stock at specific date",
    "parameters": {
        "type": "object",
        "properties": {
            "date": {
                "type":"string",
                "description":"date with format YYYY-MM_DD, which specifies value of account in a day/date",
            },
        },
        "required": ["date"],
    },
}

# def add_cash(self, value)
addCash_function = {
    "name": "add_cash",
    "description": "add specific amount (as amount of currency) to the account as cash. Note that it is not stock purchase",
    "parameters": {
        "type": "object",
        "properties": {
            "value": {
                "type":"string",
                "description":"a number that shows amount currency that should be added to the user's account",
            },
        },
        "required": ["value"],
    },
}

tools = [
    {"type": "function", "function": cash_function},
    {"type": "function", "function": buy_function},
    {"type": "function", "function": sell_function},
    {"type": "function", "function": get_price_function},
    {"type": "function", "function": get_balance_function},
    {"type": "function", "function": addCash_function},
]

In [34]:
###############################################################################
# 5) Handle Tool Calls
###############################################################################
def handle_tool_call(message):
    """
    The LLM can request to call a function in 'tools'. We parse the JSON arguments
    and run the Python function. Then we return a 'tool' message with the result.
    """
    tool_call = message.tool_calls[0]
    fn_name   = tool_call.function.name
    args      = json.loads(tool_call.function.arguments)
# get_cash []
# add_cash [value]
# portfolio_value [date]
# get_price [date, ticker]
# sell [date,ticker, shares]
# buy [date,ticker, shares]
    
    if fn_name == "get_cash":
        # No args needed
        cash_balance = sim.get_cash()
        response_content = {"cash": cash_balance}
        

    elif fn_name == "add_cash":
        value = args.get("value")
        update_value = sim.add_cash(value)
        response_content = {"update_cash": update_value}

    elif fn_name == "portfolio_value":
        date        = args.get("date")
        balance = sim.portfolio_value(date)
        response_content = {"account_balance": balance}

    elif fn_name == "get_price":
        date = args.get("date")
        ticker = args.get("ticker")
        out = sim.get_price(date, ticker)
        response_content = {"report": out}

    elif fn_name == "sell":
        date = args.get("date")
        ticker = args.get("ticker")
        shares = args.get("shares")
        rep = sim.sell(date, ticker, shares)
        response_content = {"report":rep}
    
    elif fn_name == "buy":
        date = args.get("date")
        ticker = args.get("ticker")
        shares = args.get("shares")
        rep = sim.buy(date, ticker, shares)
        response_content = {"report":rep}

    else:
        response_content = {"error": f"Unknown tool: {fn_name}"}

    return {
        "role": "tool",
        "content": json.dumps(response_content),
        "tool_call_id": tool_call.id,
    }, args


In [23]:
###############################################################################
# 6) Main Chat Function
###############################################################################
def chat(message, history):
    """
    The main chat loop that handles the conversation with the user,
    passing 'tools' definitions to the LLM for function calling.
    """
    # import pdb; pdb.set_trace()
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    try:
        response = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )

        # If the LLM requests a function call, handle it
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            print(f"[INFO] Tool call requested: {msg.tool_calls[0]}")
            tool_response, tool_args = handle_tool_call(msg)
            print(f"[INFO] Tool response: {tool_response}")

            # Add both the LLM's request and our tool response to the conversation
            messages.append(msg)
            messages.append(tool_response)

            # Re-send updated conversation to get final or next step
            response = openai.chat.completions.create(
                model=MODEL,
                messages=messages
            )

        # Return normal text response (finish_reason = "stop")
        return response.choices[0].message.content

    except Exception as e:
        print(f"[ERROR] {e}")
        return "I'm sorry, something went wrong while processing your request."

In [ ]:
###############################################################################
# 7) Launch Gradio
###############################################################################
gr.ChatInterface(fn=chat, type="messages").launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


[INFO] Tool call requested: ChatCompletionMessageToolCall(id='call_qdUMHpUzsIoZyfbHGG3Cur3w', function=Function(arguments='{"date":"2022-02-15"}', name='portfolio_value'), type='function')
[INFO] Tool response: {'role': 'tool', 'content': '{"account_balance": "The total balance in your account is 10008.06900024414"}', 'tool_call_id': 'call_qdUMHpUzsIoZyfbHGG3Cur3w'}
[INFO] Tool call requested: ChatCompletionMessageToolCall(id='call_DqOdbl4AyHeaPElZ9bM56uxJ', function=Function(arguments='{}', name='get_cash'), type='function')
[INFO] Tool response: {'role': 'tool', 'content': '{"cash": "Your account\'s cash balance is 9156.960144042969"}', 'tool_call_id': 'call_DqOdbl4AyHeaPElZ9bM56uxJ'}
[INFO] Tool call requested: ChatCompletionMessageToolCall(id='call_GwkgqCDoypZGutsFJpG6lAKX', function=Function(arguments='{"value":"5000"}', name='add_cash'), type='function')
[INFO] Tool response: {'role': 'tool', 'content': '{"update_cash": "$5000.0 is added as cash to the account. Your account\'s ca

In [ ]:
2022-02-15
AAPL